In [9]:
import os

In [15]:
# check the project working directory
%pwd

'c:\\Text-Summarizer-NLP\\research'

In [16]:
os.chdir('../')

In [17]:
%pwd

'c:\\Text-Summarizer-NLP'

In [18]:
from dataclasses import dataclass           # This is a decorator from the dataclasses module in Python that is used to automatically generate special methods in classes, such as __init__, __repr__, and __eq__.
from pathlib import Path                    # This is a class from the pathlib module which provides a convenient way to work with file system paths. It allows you to handle file paths in a platform-independent manner.

@dataclass(frozen=True)                     # This decorator converts the DataIngestionConfig class into a dataclass. The frozen=True parameter makes instances of this class immutable, meaning that once an instance is created, you cannot modify its attributes.
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [55]:
from src.Text_Summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH                                     # import the file paths stored in 'constants' folder in src
from src.Text_Summarizer.utils.common import read_yaml, create_directories       # import those util functions 


ImportError: cannot import name 'CONFIG_FILE_PATH' from 'src.Text_Summarizer.constants' (c:\Text-Summarizer-NLP\src\Text_Summarizer\constants\__init__.py)

In [51]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

NameError: name 'CONFIG_FILE_PATH' is not defined